# **Sesión 3:** Procesamiento de imagenes y extracción de características

## **Librerías**

In [24]:
import numpy as np
import cv2
import os
import imageio
from typing import List

## **Apartado C: Detección de esquinas**

El objetivo de este apartado es detectar las esquinas presentes en las imágenes de la carpeta ``data/partC/source``.

1. **Tarea C.1**. Cree una nueva capeta llamada ``partC_processed``, dentro de la carpeta  ``data/partC/source``, con el objetivo de presentar en ella los resultados de esta parte de la práctica.
2. **Tarea C.2**. Defina y ejecute los dos métodos propuestos para cargar imágenes ``imageio_load_images()`` y ``opencv_load_images()``. Observe lo que ocurre al guardar ambas imágenes usando la misma función ``cv2.imwrite()``.
3. **Tarea C.3.** Defina la función ``harris_corner_detector()``, que servirá para la posterior aplicación sobre las imágenes de trabajo. 
4. **Tarea C.4.** Aplique la función ``harris_corner_detector()`` sobre las imágenes de trabajo. Asegúrese de que las imágenes quedan guardadas como se especifica en los comentarios.
5. **Tarea C.5.** Defina la función ``shi_tomasi_corner_detection()``, que servirá para la posterior aplicación sobre las imágenes de trabajo.
6. **Tarea C.6.** Aplique la función ``shi_tomasi_corner_detection()`` sobre las imágenes de trabajo. Asegúrese de que las imágenes quedan guardadas como se especifica en los comentarios.

### **Tarea C.1**. Cree una nueva capeta llamada ``partC_processed``, dentro de la carpeta  ``data/partC/source``, con el objetivo de presentar en ella los resultados de esta parte de la práctica.

In [25]:
# TODO Create a folder to save all partA results (inside data)
folder_name = "partC_processed"
folder_path = os.path.join(os.getcwd(),folder_name)

os.makedirs(folder_path,exist_ok=True)

### **Tarea C.2**. Defina y ejecute los dos métodos propuestos para cargar imágenes ``imageio_load_images()`` y ``opencv_load_images()``. Observe lo que ocurre al guardar ambas imágenes usando la misma función ``cv2.imwrite()``.

In [26]:
# This initial part is to highlight that cv2.imshow() and cv2.imwrite() works well with previous BGR conversion
def imageio_load_images(filenames: List) -> List:
    '''
    Load images using imageio.imread function (RGB)
    '''
    return [imageio.v2.imread(filename) for filename in filenames]

def opencv_load_images(filenames: List) -> List:
    '''
    Load images cv2.imread function (BGR)
    '''
    return [cv2.imread(filename) for filename in filenames]

# TODO Create two sorted lists with the paths of all images in the data/source folder using glob

path = "../data/partC"
img_names = ["football.png","geometry.jpg","sudoku.png","tennis.png"]
source_paths = [os.path.join(path,name) for name in img_names]
imageio_images = imageio_load_images(source_paths)
opencv_images = opencv_load_images(source_paths)

# TODO Last element of both image lists is a blue tennis court, so try saving them in partA folder
cv2.imwrite(os.path.join(folder_path,"imageio_tennis.png"),imageio_images[-1])
cv2.imwrite(os.path.join(folder_path,"opencv_tennis.png"),opencv_images[-1])

True

### **Tarea C.3.** Defina la función ``harris_corner_detector()``, que servirá para la posterior aplicación sobre las imágenes de trabajo. 

In [27]:
# TODO Define Harris Corner detection function
def harris_corner_detector(image: np.array, blockSize: int, ksize: int, k: float):
    '''
    image - Input image 
    blockSize - Size of neighborhood considered for corner detection
    ksize - Aperture parameter of the Sobel derivative used
    k - Harris detector free parameter in the equation.
    '''
    # TODO Input image to Harris corner detector should be grayscale 
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    # TODO Input image to Harris corner detector should be float32 type
    gray = np.float32(gray)

    # TODO Apply Harris corner detection
    harris = cv2.cornerHarris(gray, blockSize, ksize, k)

    # Result is dilated for marking the corners, not important
    harris = cv2.dilate(harris, None)
    # TODO Threshold for an optimal value of 1% of maximal R value
    # If pixel value > 1% max value, yo should to hightlight this as a red corner
    result = image.copy()

    # Umbral: marcar en rojo las esquinas
    threshold = 0.01 * harris.max()
    result[harris > threshold] = [0, 0, 255] 

    return result


### **Tarea C.4.** Aplique la función ``harris_corner_detector()`` sobre las imágenes de trabajo. Asegúrese de que las imágenes quedan guardadas como se especifica en los comentarios.

In [28]:
# This section is designed to to change corner detection parameters for each image
# We want to save processed image in path: Lab3/data/partC-D-E/Harris_{save_name}.jpg
blockSize = 2
ksize = 3
k = 0.04

# First image
save_name = "geometry"
save_name = "Harris_"+ save_name
# TODO Copy first original image
image = opencv_images[1]
# TODO Apply Harris Corner Detection
harris_image = harris_corner_detector(image, blockSize, ksize, k)
# TODO Save final image in partA folder
cv2.imwrite(f"{os.path.join(folder_path,save_name)}.jpg",harris_image)

# Second image
save_name = "football"
save_name = "Harris_"+ save_name
# TODO Copy second original image
image = opencv_images[0]
# TODO Apply Harris Corner Detection
harris_image = harris_corner_detector(image, blockSize, ksize , k)
# TODO Save final image in partA folder
cv2.imwrite(f"{os.path.join(folder_path,save_name)}.png",harris_image)

True

### **Tarea C.5.** Defina la función ``shi_tomasi_corner_detection()``, que servirá para la posterior aplicación sobre las imágenes de trabajo.

In [29]:
# TODO Define Shi-Tomasi corner detection function
def shi_tomasi_corner_detection(image: np.array, maxCorners: int, qualityLevel:float, minDistance: int, corner_color: tuple, radius: int):
    '''
    image - Input image
    maxCorners - Maximum number of corners to return. 
                 If there are more corners than are found, the strongest of them is returned. 
                 maxCorners <= 0 implies that no limit on the maximum is set and all detected corners are returned
    qualityLevel - Parameter characterizing the minimal accepted quality of image corners. 
                   The parameter value is multiplied by the best corner quality measure, which is the minimal eigenvalue or the Harris function response. 
                   The corners with the quality measure less than the product are rejected. 
                   For example, if the best corner has the quality measure = 1500, and the qualityLevel=0.01 , then all the corners with the quality measure less than 15 are rejected
    minDistance - Minimum possible Euclidean distance between the returned corners
    corner_color - Desired color to highlight corners in the original image
    radius - Desired radius (pixels) of the circle
    '''
    # TODO Input image to Tomasi corner detector should be grayscale 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # TODO Apply cv2.goodFeaturesToTrack function
    corners = cv2.goodFeaturesToTrack(gray, maxCorners=maxCorners, qualityLevel=qualityLevel, minDistance=minDistance)

    # TODO Corner coordinates conversion to integers
    if corners is not None:
        corners = np.int0(corners)
        for corner in corners:
            x, y = corner.ravel()
            cv2.circle(image, (x, y), radius, corner_color, thickness=-1)

    return image


### **Tarea C.6.** Aplique la función ``shi_tomasi_corner_detection()`` sobre las imágenes de trabajo. Asegúrese de que las imágenes quedan guardadas como se especifica en los comentarios.

In [30]:
# This section is designed to to change corner detection parameters for each image
# We want to save processed image in path: Lab3/data/partC-D-E/Shi-Tomasi_{save_name}.jpg

# First image - Purple corners and radius = 4
maxCorners = 100
qualityLevel = 0.01
minDistance = 10
radius = 4


save_name = "geometry"
save_name = "Shi-Tomasi_"+ save_name
# TODO Purple color in adequate color space
purple_color = (255, 0, 255)
# TODO Copy first original image
image = opencv_images[1]
# TODO Apply Shi-Tomasi corner detection
tomasi_image = shi_tomasi_corner_detection(image, maxCorners, qualityLevel, minDistance,purple_color, radius)
# TODO Save final image in partC folder
cv2.imwrite(f"{os.path.join(folder_path,save_name)}.jpg",tomasi_image)

# Second image - Orange corners and radius = 4
save_name = "football"
save_name = "Shi-Tomasi_"+ save_name
# TODO Orange color in adequate color space
orange_color = (0, 165, 255)
# TODO Copy second original image
image = opencv_images[0]
tomasi_image = shi_tomasi_corner_detection(image, maxCorners, qualityLevel, minDistance, orange_color, radius)
# TODO Save final image in partC folder
cv2.imwrite(f"{os.path.join(folder_path,save_name)}.png",tomasi_image)

C:\Users\rayne\AppData\Local\Temp\ipykernel_2860\3563165958.py:23: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  corners = np.int0(corners)


True

### **Pregunta C.1:** Realice la detección de esquinas en las otras dos imágenes de la carpeta ``data/partC-D-E/source`` (cuyos nombres de guardado han de ser "sudoku" y "tennis") con el método de Harris

In [31]:
# TODO Homework
# This section is designed to to change corner detection parameters for each image
# We want to save processed image in path: Lab3/data/partC-D-E/Harris_{save_name}.jpg
blockSize = 2
ksize = 3
k = 0.04

# First image
save_name = "sudoku"
save_name = "Harris_"+ save_name
# TODO Copy first original image
image = opencv_images[2]
# TODO Apply Harris Corner Detection
harris_image = harris_corner_detector(image, blockSize, ksize, k)
# TODO Save final image in partA folder
cv2.imwrite(f"{os.path.join(folder_path,save_name)}.jpg",harris_image)

# Second image
save_name = "tennis"
save_name = "Harris_"+ save_name
# TODO Copy second original image
image = opencv_images[-1]
# TODO Apply Harris Corner Detection
harris_image = harris_corner_detector(image, blockSize, ksize , k)
# TODO Save final image in partA folder
cv2.imwrite(f"{os.path.join(folder_path,save_name)}.png",harris_image)

True

### **Pregunta C.2:** Realice la detección de esquinas en las otras dos imágenes de la carpeta ``data/partC-D-E/source`` (cuyos nombres de guardado han de ser "sudoku" y "tennis") con el método de Shi-Tomasi

In [33]:
# TODO Homework

maxCorners = 100
qualityLevel = 0.01
minDistance = 10
radius = 4


save_name = "sudoku"
save_name = "Shi-Tomasi_"+ save_name
# TODO Purple color in adequate color space
purple_color = (255, 0, 255)
# TODO Copy first original image
image = opencv_images[2]
# TODO Apply Shi-Tomasi corner detection
tomasi_image = shi_tomasi_corner_detection(image, maxCorners, qualityLevel, minDistance,purple_color, radius)
# TODO Save final image in partC folder
cv2.imwrite(f"{os.path.join(folder_path,save_name)}.jpg",tomasi_image)

# Second image - Orange corners and radius = 4
save_name = "tennis"
save_name = "Shi-Tomasi_"+ save_name
# TODO Orange color in adequate color space
orange_color = (0, 165, 255)
# TODO Copy second original image
image = opencv_images[-1]
tomasi_image = shi_tomasi_corner_detection(image, maxCorners, qualityLevel, minDistance, orange_color, radius)
# TODO Save final image in partC folder
cv2.imwrite(f"{os.path.join(folder_path,save_name)}.png",tomasi_image)

C:\Users\rayne\AppData\Local\Temp\ipykernel_2860\3563165958.py:23: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  corners = np.int0(corners)


True